# Task 2: Recommendation Engine

This notebook provides a very basic example for the notebook you are expected to submit for Task 2 of the Final Project. The main purpose is that we can try different examples to get a better sense of your approach. Compared to Task 1 (Kaggle Competition), we don't have any objective means to evaluate the recommendations. 

Some general comments:
* You can import any data you need. This particularly includes your cleaned version of the Used Cars dataset; there's no need to show the data cleaning / preprocessing steps in this notebook.
* You can also import your code in form of external Python (.py) script. You're actually encouraged to do so to keep this notebook light and uncluttered.
* Please consider this notebook as an example and not to set specific requirements. As long there is a section where we can easily test your solution, it should be fine.

## Setting up the Notebook

In [11]:
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import rankdata
from scipy import spatial

import numpy as np
import pandas as pd

In [12]:
def prepare_features(data, numerical_features, string_features):
    data.loc[data['make'].isnull(), 'make'] = data['title'].map(lambda x: x.split(" ")[0])
    data['make'] = data['make'].str.lower()
    data[string_features] = data[string_features].fillna("")
    data[numerical_features] = preprocessing.MinMaxScaler().fit_transform(data[numerical_features])
    numerical_mean = data[numerical_features].mean()
    data[numerical_features] = data[numerical_features].fillna(numerical_mean)
    return data

# Derive based on 99% of data range
feature_range = {
    "price_std": [20000, 200000],
    "manufactured_std": [1990, 2021],
#     "mileage_std": [30, 200000]
}

def limit_outliers(data):
    for feature, min_max in feature_range.items():
        min_val, max_val = min_max[0], min_max[1]
        data.loc[data[feature] < min_val, feature] = min_val 
        data.loc[data[feature] > max_val, feature] = max_val 
    return data 

def generate_description(row, feature_names):
    return " ".join([row[feature] for feature in feature_names])

## Load the Data

For this example, we use a simplified version of the dataset with only 100 data sample, each with only 6 features

In [14]:
df = pd.read_csv('../data/train.csv')

feature_names = ["make", "description", "transmission", "type_of_vehicle", "features"]
numerical_features = ["price", "power", "manufactured"]
std_numerical_features = [f + '_std' for f in numerical_features]

df[std_numerical_features] = df[numerical_features]
df = limit_outliers(df)
df = prepare_features(df, std_numerical_features, feature_names)

description_ls = [generate_description(row, feature_names)
                  for _, row in df.iterrows()]

df = pd.concat((df, pd.DataFrame(description_ls, columns=['desc_concat'])), axis=1)

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(description_ls)

tfidf_score_arr = np.array(vectors.todense())

## Computing the Top Recommendations

The method `get_top_recommendations()` shows an example of how to get the top recommendations for a given data sample (data sample = row in the dataframe of the dataset). The input is a row from the dataset and a list of optional input parameters which will depend on your approach; `k` is the number of returned recommendations seems useful, though.

The output should be a `pd.DataFrame` containing the recommendations. The output dataframe should have the same columns as the row + any additional columns you deem important (e.g., any score or tags that you might want to add to your recommendations).

In principle, the method `get_top_recommendations()` may be imported from a external Python (.py) script as well.

In [15]:
def get_top_recommendations(row, train_df, string_features, numerical_features, vectorizer, tfidf_score_arr, **kwargs) -> pd.DataFrame:
    
    #####################################################
    ## Initialize the required parameters
    
    # The number of recommendations seem recommended
    # Additional input parameters are up to you
    k = 1
    
    # Extract all **kwargs input parameters
    # and set the used paramaters (here: k)
    for key, value in kwargs.items():
        if key == 'k':
            k = value
            
       
    #####################################################
    ## Compute your recommendations
    question = generate_description(row, string_features)
    
    tfidf_question = vectorizer.transform([question])

    query_mask = np.where(np.array(tfidf_question.todense())>0, 1, 0)

    doc_score = np.sum(query_mask * tfidf_score_arr, axis=1)
    
    row_feature = row[numerical_features].to_numpy()
    features_numpy = train_df[numerical_features].to_numpy()
    numerical_similarity = cosine_similarity([row_feature], features_numpy)[0]
    
    numerical_rank = rankdata(numerical_similarity, method="min")
    doc_rank = rankdata(doc_score, method="min")
    final_score = 2 * (train_df.shape[0] + 1) - (numerical_rank + doc_rank)

    doc_rank_idx = final_score.argsort()
    
    doc_rank_idx = np.delete(doc_rank_idx, np.where(doc_rank_idx == row.name))
    doc_rank_idx = doc_rank_idx[:k]

    doc_score = doc_score[doc_rank_idx]
    
    df_result = train_df.loc[doc_rank_idx]
    
    df_result['doc_score'] = doc_score
    df_result['final_score'] = final_score[doc_rank_idx]
    # Return the dataset with the k recommendations
    return df_result


## Testing the Recommendation Engine

This will be the main part of your notebook to allow for testing your solutions. Most basically, for a given listing (defined by the row id in your input dataframe), we would like to see the recommendations you make. So however you set up your notebook, it should have at least a comparable section that will allow us to run your solution for different inputs.

### Pick a Sample Listing as Input

In [16]:
features = ["make", "model", "description", "transmission", "type_of_vehicle", "features", "price", "power", "manufactured"]

In [17]:
# Pick a row id of choice
# row_id = 10
row_id = 20
# row_id = 30
# row_id = 70
# row_id = 50

# Get the row from the dataframe (an valid row ids will throw an error)
row = df.iloc[row_id]

# Just for printing it nicely, we create a new dataframe from this single row
pd.DataFrame([row])[features]

,make,model,description,transmission,type_of_vehicle,features,price,power,manufactured
20,volkswagen,scirocco,nothing like a 2.0tsi ea888 engine in a sciroc...,auto,sports car,"powerful yet economical 2.0tsi ea888 engine, p...",79000.0,147.0,2009.0


## Compute and Display the recommendations

Since the method `get_top_recommendations()` returns a `pd.DataFrame`, it's easy to display the result.

In [18]:


k = 10

df_recommendations = get_top_recommendations(row, df, feature_names, std_numerical_features, vectorizer, tfidf_score_arr, k=k)
df_recommendations.head(k)[features]



,make,model,description,transmission,type_of_vehicle,features,price,power,manufactured
12961,volkswagen,scirocco,properly done up scirocco 2.0tsi with chilli r...,auto,sports car,"2.0tsi engine, reliable 6-speed dq250 wet clut...",82300.0,147.0,2009.0
11655,volkswagen,scirocco,guaranteed cheapest! 100% loan and trade in av...,auto,sports car,smooth and powerful 2.0 tsi engine mated with ...,85600.0,155.0,2010.0
16693,volkswagen,golf,82999,auto,sports car,powerful 2.0l 4-cylinders 16 valve tsi turboch...,80300.0,155.0,2009.0
15923,volkswagen,golf,100% loan/high trade in available! genuine low...,auto,sports car,powerful and reliable 2.0l turbo-charged engin...,84700.0,155.0,2009.0
13913,volkswagen,scirocco,rare stock condition vw scirroco for sale! tre...,auto,sports car,2.0 tsi ea888 engine with 6 speed dsg wet clut...,78600.0,155.0,2009.0
4837,volkswagen,golf,0,auto,sports car,powerful and reliable 2.0l turbocharged engine...,76800.0,155.0,2010.0
825,volkswagen,golf,one of the finest compact sports car suitable ...,auto,sports car,"2.0l turbo charged engine, 6 speed dsg gearbox...",86800.0,155.0,2009.0
9194,volkswagen,golf,nicely done up mk6 gti in the market. stage 2 ...,auto,sports car,powerful and reliable 2.0l turbocharged engine...,83500.0,155.0,2009.0
9004,bmw,325i,"consignment by owner, viewing by appointment o...",auto,sports car,fully stock condition. view specs of the bmw 3...,81200.0,160.0,2009.0
13454,audi,a5,original colour till now. well maintained unit...,auto,sports car,"2.0l turbocharged engine producing, 7 speed du...",86700.0,155.0,2010.0
